In [5]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division

In [6]:
# form mzp6.ipnyb
def draw_system(method, t_0, y_0s, approx, exact = None, labels=None, step = 0.1):
    count = np.ceil(10.0/step)
    X = np.linspace(t_0,t_0+10,count)
    Y = np.zeros(shape=(len(approx),X.shape[0]))
    if exact:
        RY = np.asarray([y(X) if y is not None else None for y in exact])
    Y[:,0] = np.asarray(y_0s)
    
    for i in np.arange(1,count):
        Y_updates = method(tn=X[i-1],yn=Y[:,i-1],functions=approx,h=step)
        if isinstance(Y_updates,tuple):
            Y[:,i] = Y[:,i-1] + Y_updates[1]
        else:
            Y[:,i] = Y[:,i-1] + Y_updates
    
    figure_count = len(approx)
    if exact:
        figure_count += len(filter(lambda x: x is not None,exact))
    figure = 1
    plt.figure(figsize = (9,figure_count*3))
    for i in range(len(approx)):
        ax = plt.subplot(figure_count,1,figure)
        if labels and labels[i]:
            ax.set_title(labels[i])
        plt.plot(X,Y[i], color = 'blue', label ='Approx')
        if exact and exact[i]:
            plt.plot(X,RY[i], color = 'red', linestyle='--', label='Exact')
        plt.legend(loc = 'upper left')
        figure += 1

        if exact and exact[i]:
            plt.subplot(figure_count,1,figure)
            plt.plot(X,np.abs(RY[i]-Y[i]), color = 'blue', label= 'Global error')
            plt.legend(loc = 'upper left')
            figure+=1
            
    plt.show()

In [7]:
# form mzp6.ipnyb
class SystemSolver():
    def __init__(self,updates = None):
        if isinstance(updates,list):
            max_len = max([len(x) for x in updates])
            [x.extend([0]*(max_len-len(x))) for x in updates]
            self.updates = np.asarray(updates, dtype = np.float64)
        else:
            self.updates = updates
        
    @property
    def update_count(self):
        return min(self.updates.shape)-1
    
    def __call__(self,tn,yn,functions,h):
        k = np.zeros(shape=(self.update_count,len(functions)))
        for i in range(self.update_count):
            a = self.updates[i,0]
            b = self.updates[i,1:]
            upd = np.sum(b[:,np.newaxis]*k,axis = 0)
            for j in range(len(functions)):
                k[i,j]= h * functions[j](tn+a*h, yn+upd)
        c = self.updates[self.update_count,1:]
        if self.updates.shape[0]>self.updates.shape[1]:
            c_= self.updates[self.update_count+1,1:]
            return np.sum(c[:,np.newaxis]*k, axis = 0), np.sum(c_[:,np.newaxis]*k, axis = 0)
        return np.sum(c[:,np.newaxis]*k, axis = 0)

In [8]:
fehlberg_4_5 = SystemSolver([[0],
                            [1/4,1/4],
                            [3/8,3/32,9/32],
                            [12/13,1932/2197,-7200/2197,7296/2197],
                            [1,439/216,-8,3680/513,-845/4104],
                            [1/2,-8/27,2,-3544/2565,1859/4104,-11/40],
                            [0,25/216,0,1408/2565,2197/4104,-1/5,0],
                            [0,16/135,0,6656/12825,28561/56430,-9/50,2/55]])
fehlberg_4_5.ranks = (4,5)

In [9]:
def get_spring(start,end,count,size = 1.):
    start = np.asarray(start)
    end = np.asarray(end)
    length = np.sqrt(np.sum((end-start)**2))
    flat_space = length/6.
    x = np.linspace(flat_space,length-flat_space,count*4+1)
    length_needed = 2*np.pi*count 
    s = size*np.sin((length_needed/(length-flat_space*2))*x)
    rotations = (end-start)/length
    rotation_matrix = np.array([[rotations[0],-rotations[1]],[rotations[1],rotations[0]]])
    first_flat = np.array([[0],[0]])
    second_flat = np.array([[length],[0]])
    return rotation_matrix.dot(np.hstack((first_flat, np.array([x,s]), second_flat))) + start[:,np.newaxis]

Dwie spreżyny

In [28]:
k_1 = 4
k_2 = 10
L = 3
m = 3

In [29]:
x_0 = 0
v_0 = 3

In [146]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1 - k_2)/m*ys[0]
]

In [31]:
draw_system(fehlberg_4_5,
            0.,
            y_0s=[x_0,v_0],
            approx = functions,
            labels = ["x(t)","v(t)"])

In [32]:
Y = np.zeros(shape=(2,))

In [35]:
def double_spring(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    spring1, = ax.plot([],[])
    spring2, = ax.plot([],[])
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], marker='o', markersize = 10.)
    
    def init():
        global Y
        Y = y_0s
        spring1.set_data([],[])
        spring2.set_data([],[])
        time_text.set_text('')
        mass.set_data([],[])
        
        return spring1, spring2, time_text, mass
    
    def animate(t):
        global Y
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        new_x = Y[0]
        spring = get_spring([-10,0],[new_x,0],6)
        spring1.set_data(spring[0],spring[1])
        spring = get_spring([new_x,0],[10,0],6)
        spring2.set_data(spring[0],spring[1])
        mass.set_data([new_x],[0])

        time_text.set_text(time_template % (t*step))
        return spring1, spring2, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [147]:
k_1 = 5
k_2 = 10
L = 3
m = 3

In [148]:
x_0 = 2
v_0 = 3

In [149]:
double_spring(method=fehlberg_4_5,
               y_0s=[x_0,v_0],
               approx= functions,
               step=0.05,
              filename='double_spring_1')

In [142]:
def triple_spring(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    spring1, = ax.plot([],[])
    spring2, = ax.plot([],[])
    spring3, = ax.plot([],[])
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    points = (-15,-5,5,15)
    
    def init():
        global Y
        Y = y_0s
        spring1.set_data([],[])
        spring2.set_data([],[])
        spring3.set_data([],[])
        time_text.set_text('')
        mass.set_data([],[])
        
        return spring1, spring2, spring3, time_text, mass
    
    def animate(t):
        global Y
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        fixed_1 = points[0]
        mass_1 = points[1]+Y[0]
        mass_2 = points[2]+Y[2]
        fixed_2 = points[3]
        
        spring = get_spring([fixed_1,0],[mass_1,0],6)
        spring1.set_data(spring[0],spring[1])
        
        spring = get_spring([mass_1,0],[mass_2,0],6)
        spring2.set_data(spring[0],spring[1])
        
        spring = get_spring([mass_2,0],[fixed_2,0],6)
        spring3.set_data(spring[0],spring[1])
        
        mass.set_data([mass_1,mass_2],[0,0])

        time_text.set_text(time_template % (t*step))
        return spring1, spring2, spring3, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [126]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1*ys[0]+k_2*(ys[0]-ys[2]))/-m_1,
    lambda t,ys: ys[3],
    lambda t,ys: (k_3*ys[2]+k_2*(ys[2]-ys[0]))/-m_2
]

In [127]:
k_1 = 2
k_2 = 2
k_3 = 2
m_1 = .2
m_2 = .2

In [134]:
x_1_0 = 3
v_1_0 = 0
x_2_0 = 0
v_2_0 = 0

In [140]:
draw_system(fehlberg_4_5,
            0.,
            y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
            approx = functions,
            labels = ["x_1(t)","v_1(t)","x_2(t)","v_2(t)"])

In [144]:
triple_spring(method=fehlberg_4_5,
              y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
              approx= functions,
              step=0.05,
              filename = 'triple_spring_1')

Z oporami ($\psi(v)$) zależnymi od prędkości

$\psi(v) = c  v$

In [172]:
c = 1

In [208]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1*ys[0]+k_2*(ys[0]-ys[2]))/-m_1 - c*ys[1],
    lambda t,ys: ys[3],
    lambda t,ys: (k_3*ys[2]+k_2*(ys[2]-ys[0]))/-m_2 - c*ys[3]
]

In [209]:
triple_spring(method=fehlberg_4_5,
              y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
              approx= functions,
              step=0.05,
              filename='triple_spring_2')

$\psi(v) = sgn(v) c  v^2$

In [227]:
c = 0.1

In [207]:
x_1_0 = 20
v_1_0 = 0
x_2_0 = -10
v_2_0 = 0

In [228]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1*ys[0]+k_2*(ys[0]-ys[2]))/-m_1 - c*np.sign(ys[1])*ys[1]**2,
    lambda t,ys: ys[3],
    lambda t,ys: (k_3*ys[2]+k_2*(ys[2]-ys[0]))/-m_2 - c*np.sign(ys[3])*ys[3]**2
]

In [233]:
triple_spring(method=fehlberg_4_5,
              y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
              approx= functions,
              step=0.05,
              filename = 'triple_spring_3')